In [1]:
import pandas as pd
import datetime
import seaborn as sb
import time
#from ipywidgets import interact, fixed
from libs.CovidTimeseriesModel import CovidTimeseriesModel 
from libs.CovidTimeseriesModelSIR import CovidTimeseriesModelSIR
from libs.CovidDatasets import CovidDatasets
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [8]:
%load_ext autoreload
%autoreload 2

In [2]:
Dataset = CovidDatasets()

def model_state(country, state, interventions=None):
   
    ## Constants
    start_time = time.time()
    HOSPITALIZATION_RATE = .0727
    HOSPITALIZED_CASES_REQUIRING_ICU_CARE = .1397
    TOTAL_INFECTED_PERIOD = 12
    MODEL_INTERVAL = 4
    r0 = 2.4
    POP = Dataset.get_population_by_country_state(country, state)
    # Pack all of the assumptions and parameters into a dict that can be passed into the model
    MODEL_PARAMETERS = {
        # Pack the changeable model parameters
        'timeseries': Dataset.get_timeseries_by_country_state(country, state, MODEL_INTERVAL),
        'beds': Dataset.get_beds_by_country_state(country, state),
        'population': POP,
        'projection_iterations': 25, # Number of iterations into the future to project
        'r0': r0,
        'interventions': interventions,
        'hospitalization_rate': HOSPITALIZATION_RATE,
        'case_fatality_rate': .0109341104294479,
        'hospitalized_cases_requiring_icu_care': HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
        # Assumes that anyone who needs ICU care and doesn't get it dies
        'case_fatality_rate_hospitals_overwhelmed': HOSPITALIZATION_RATE * HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
        'hospital_capacity_change_daily_rate': 1.05,
        'max_hospital_capacity_factor': 2.07,
        'initial_hospital_bed_utilization': .6,
        'model_interval': 4, # In days
        'total_infected_period': 12, # In days
        'rolling_intervals_for_current_infected': int(round(TOTAL_INFECTED_PERIOD / MODEL_INTERVAL, 0)),
        'estimated_new_cases_per_death': 32,
        'estimated_new_cases_per_confirmed': 20
    }
    return CovidTimeseriesModelSIR().forecast_region(model_parameters=MODEL_PARAMETERS)

r0 = 2.4

INTERVENTIONS = [
    None, 
    {
        datetime.date(2020, 3, 23): 1.3,
        datetime.date(2020, 4, 20): 1.1,
        datetime.date(2020, 5, 22): 0.8,
        datetime.date(2020, 6, 23): r0
    },
    {
        datetime.date(2020, 3, 23): 1.7,
        datetime.date(2020, 6, 23): r0
    },
    {
        datetime.date(2020, 3, 23): 1.3,
        datetime.date(2020, 3, 31): 0.3,
        datetime.date(2020, 4, 28): 0.2,
        datetime.date(2020, 5,  6): 0.1,
        datetime.date(2020, 5, 10): 0.35,
        datetime.date(2020, 5, 18): r0
    }
]

In [3]:
# Dataset.get_timeseries_by_country_state('USA', 'CA', 1)[['date','cases','deaths']]

In [48]:
model_state('USA', 'TX', INTERVENTIONS[0])

2020-06-15 00:00:00


,Date,Timestamp,R,Effective R.,Beg. Susceptible,New Inf < C,New Inf < D,New Inf.,Curr. Inf.,Recov. or Died,End Susceptible,Actual Reported,Pred. Hosp.,Cum. Inf.,Cum. Deaths,Avail. Hosp. Beds
0,2020-03-07,1583564400.00,2.40,2.40,28995873.00,0,0,8,8.00,0.00,28995873.00,0,0,0.00,nan,0
1,2020-03-08,1583650800.00,2.40,2.40,28995795.24,75,75,83,83.06,2.70,28995795.24,0,0,75.06,2.70,0
2,2020-03-09,1583733600.00,2.40,2.40,28994988.02,779,779,862,862.25,30.73,28994988.02,0,0,854.25,30.73,0
3,2020-03-10,1583820000.00,2.40,2.40,28986610.18,8087,8087,8949,8949.14,321.68,28986610.18,0,0,8941.14,321.68,0
4,2020-03-11,1583906400.00,2.40,2.40,28899922.29,83672,83672,92622,92621.56,3337.15,28899922.29,0,0,92613.56,3337.15,0
5,2020-03-12,1583992800.00,2.40,2.40,28030161.34,838995,838995,931617,931616.89,34102.77,28030161.34,0,0,931608.89,34102.77,0
6,2020-03-13,1584079200.00,2.40,2.40,21387489.71,6370357,6370357,7301974,7301973.94,306417.35,21387489.71,0,0,7301965.94,306417.35,0
7,2020-03-14,1584165600.00,2.40,2.40,6372039.95,13796328,13796328,21098301,21098301.46,1525539.59,6372039.95,0,0,21098293.46,1525539.59,0
8,2020-03-15,1584252000.00,2.40,2.40,881761.53,3499074,3499074,24597375,24597375.05,3516744.42,881761.53,0,0,24597367.05,3516744.42,0
9,2020-03-16,1584338400.00,2.40,2.40,117762.63,-1262945,-1262945,23334430,23334430.30,5543688.07,117762.63,0,0,23334422.30,5543688.07,0
